# auth


In [1]:

import json
import os


from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


# env

In [2]:
import os, sys, logging
import warnings
warnings.filterwarnings("ignore")
PROJECT = "csi"
WORKDIR = "/content/icompete_csi"
DATADIR = os.path.join(WORKDIR, 'data')
MODELNAME="CSI"
NUM = int(1e16)

sys.path.insert(0, WORKDIR)
os.environ['PYTHONPATH']=WORKDIR
!mkdir -p {DATADIR}

os.chdir('/content')


## code

In [3]:
!cd /content && rm -rf icompete_csi && git clone https://github.com/world2vec/icompete_csi.git

Cloning into 'icompete_csi'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 19 (delta 2), reused 16 (delta 2), pack-reused 0
Unpacking objects: 100% (19/19), done.


# data

In [4]:
!mkdir -p {DATADIR}/{PROJECT}
!cp  /content/drive/My\ Drive/dl/{PROJECT}/data/H_4T4R.mat {DATADIR}/{PROJECT}

In [5]:
print(DATADIR)

/content/icompete_csi/data


In [6]:
ls

drive/  icompete_csi/  sample_data/


# requirements

In [7]:

!pip install psutil==5.7.0 tqdm==4.45.0
!cd {WORKDIR}/{PROJECT} && ./setup.sh > /tmp/install.log




     |████████████████████████████████| 450kB 16.3MB/s 
     |████████████████████████████████| 61kB 8.9MB/s 
  Created wheel for psutil: filename=psutil-5.7.0-cp37-cp37m-linux_x86_64.whl size=276420 sha256=047e3ff74ba4d97e2e550c1fcbdb2c822a75bd37b3e85e18a0a168d1dafbad2b
  Stored in directory: /root/.cache/pip/wheels/d7/69/b4/3200b95828d1f0ddb3cb5699083717f4fdbd9b4223d0644c57
Successfully built psutil
  Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


tcmalloc: large alloc 1147494400 bytes == 0x565079552000 @  0x7f9c3c8b9615 0x565040cf706c 0x565040dd6eba 0x565040cf9e8d 0x565040deb99d 0x565040d6dfe9 0x565040d68b0e 0x565040cfb77a 0x565040d6de50 0x565040d68b0e 0x565040cfb77a 0x565040d6a86a 0x565040dec7c6 0x565040d69ee2 0x565040dec7c6 0x565040d69ee2 0x565040dec7c6 0x565040d69ee2 0x565040dec7c6 0x565040e6e431 0x565040dcf049 0x565040d39c84 0x565040cfa8e9 0x565040d6eade 0x565040cfb69a 0x565040d69a45 0x565040d68e0d 0x565040cfb77a 0x565040d69a45 0x565040cfb69a 0x565040d69a45
tcmalloc: large alloc 1434370048 bytes == 0x5650bdba8000 @  0x7f9c3c8b9615 0x565040cf706c 0x565040dd6eba 0x565040cf9e8d 0x565040deb99d 0x565040d6dfe9 0x565040d68b0e 0x565040cfb77a 0x565040d6de50 0x565040d68b0e 0x565040cfb77a 0x565040d6a86a 0x565040dec7c6 0x565040d69ee2 0x565040dec7c6 0x565040d69ee2 0x565040dec7c6 0x565040d69ee2 0x565040dec7c6 0x565040e6e431 0x565040dcf049 0x565040d39c84 0x565040cfa8e9 0x565040d6eade 0x565040cfb69a 0x565040d69a45 0x565040d68e0d 0x565040cf

# tpu

In [8]:
import tensorflow as tf

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
    USE_TPU = "-use_tpu"
except ValueError:
    USE_TPU = ""
if USE_TPU!="":
    #mu.run_cmd("curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py")
    #mu.run_cmd("python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev")
    !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
    !python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev >/tmp/xla.log
    import torch_xla.core.xla_model as xm
    import torch_xla.distributed.parallel_loader as pl
    import torch_xla.distributed.xla_multiprocessing as xmp

# pretrain

In [11]:
os.chdir(os.path.join(WORKDIR, PROJECT))

BATCHSIZE = 512
KFID = 0
KN = 10
ENCDIM = 128
PQBIT=4
QBIT=4
WS=10000
LDS=100000
LR=8e-5
ELAYER=7
DLAYER=7
NHEAD=16
DEMODEL=544
DDMODEL=544
EFFD=2104
DFFD=2104




MINLOSS=0.085
MODELNAME="CSIPlus_p4geenc128ff2104q4e7d7h16d544_kf0b512abs1w1000lr1e4dp0ema999tqbest"


!python train.py -m kf  -kfid {KFID} -kn {KN} -ms {MODELNAME} -num {NUM}  -save  -save_best $USE_TPU   -n_es_epoch 20 -n_lr_warmup_step {WS} -n_init_epoch 0 -bs {BATCHSIZE}  -abs 1 -ema 0.999   -xla_procs 1  -train_quantize  \
-lr {LR}  -n_lr_decay_step {LDS} -lr_decay_rate 0.2  -dp 0 -enc_dim {ENCDIM} -n_q_bit {QBIT} -d_eff {EFFD} -d_dff {DFFD} -n_e_layer {ELAYER} -n_d_layer {DLAYER} -n_ehead {NHEAD}  -n_dhead {NHEAD} -d_emodel {DEMODEL}  -d_dmodel {DDMODEL} \
-activation gelu  -save_half -use_fp16  -es 1e-5 -min_loss {MINLOSS}  -epochs 57 # -n_epoch_step 10 
!ls -ltrh ../data/{MODELNAME}_KF{KFID}/*

tcmalloc: large alloc 1843208192 bytes == 0x55a72449a000 @  0x7f0b8b2091e7 0x55a71ff58f48 0x55a71ff239c7 0x7f0ac890fb32 0x7f0ac8b2a973 0x7f0ac8b2f0bd 0x7f0ac8b328ee 0x7f0ac8b3a5c0 0x7f0ac8b2c5f5 0x55a71ff27050 0x55a72001899d 0x55a71ff9afe9 0x55a71ff95b0e 0x55a71ff2877a 0x55a71ff96c9e 0x55a71ff95b0e 0x55a71ff2877a 0x55a71ff96c9e 0x55a71ff95b0e 0x55a71ff2877a 0x55a71ff9ae50 0x55a71ff2869a 0x55a71ff9ae50 0x55a71ff95b0e 0x55a71ff2877a 0x55a71ff96a45 0x55a71ff95b0e 0x55a71ff95813 0x55a72005f592 0x55a72005f90d 0x55a72005f7b6
tcmalloc: large alloc 1843200000 bytes == 0x55a79228e000 @  0x7f0b8b2091e7 0x7f0b88aed631 0x7f0b88b54680 0x7f0b88be21a7 0x55a71ff27050 0x55a72001899d 0x55a71ff9afe9 0x55a71ff2869a 0x55a71ff9ae50 0x55a71ff95b0e 0x55a71ff2877a 0x55a71ff96a45 0x55a71ff95b0e 0x55a71ff95813 0x55a72005f592 0x55a72005f90d 0x55a72005f7b6 0x55a720037103 0x55a720036dac 0x7f0b89ff3bf7 0x55a720036c8a
2021-04-01 06:30:30,620 - INFO - util -   num is 600000
tcmalloc: large alloc 1843200000 bytes == 0x

# fintuning

In [ ]:
os.chdir(os.path.join(WORKDIR, PROJECT))

BATCHSIZE = 512
KFID = 0
KN = 10
ENCDIM = 128
PQBIT=4
QBIT=3
WS=1000
LDS=100000
LR=3e-5
ELAYER=7
DLAYER=7
NHEAD=16
DEMODEL=544
DDMODEL=544
EFFD=2104
DFFD=2104


MINLOSS=0.085
MODELNAME="CSIPlus_p4geenc128ff2104q3e7d7h16d544_kf0b512abs1w1000lr1e4dp0ema999tqbest"
!mkdir -p ../data/{MODELNAME}_KF{KFID}
!cp ../data/CSIPlus_p{PQBIT}geenc{ENCDIM}ff{EFFD}q4e{ELAYER}d{DLAYER}h{NHEAD}d{DEMODEL}_kf0b512abs1w1000lr1e4dp0ema999tqbest_KF{KFID}/cfg.json ../data/{MODELNAME}_KF{KFID}/

!python train.py -m kf  -kfid {KFID} -kn {KN} -ms {MODELNAME} -num {NUM}  -save  -save_best $USE_TPU   -n_es_epoch 20 -n_lr_warmup_step {WS} -n_init_epoch 0 -bs {BATCHSIZE}  -abs 1 -ema 0.9995   -xla_procs 1  -train_quantize  \
-lr {LR}  -n_lr_decay_step {LDS} -lr_decay_rate 0.2  -dp 0 -enc_dim {ENCDIM} -n_q_bit {QBIT} -d_eff {EFFD} -d_dff {DFFD} -n_e_layer {ELAYER} -n_d_layer {DLAYER} -n_ehead {NHEAD}  -n_dhead {NHEAD} -d_emodel {DEMODEL}  -d_dmodel {DDMODEL} \
-activation gelu  -save_half -use_fp16  -es 1e-5 -min_loss {MINLOSS}  -epochs 113 -restore  # -n_epoch_step 10 
!ls -ltrh ../data/{MODELNAME}_KF{KFID}/*
!cp -r {DATADIR}/{MODELNAME}* "{WORKDIR}/drive/My Drive/dl/{PROJECT}/data" && date

tcmalloc: large alloc 1843208192 bytes == 0x5645895dc000 @  0x7f0b122cf1e7 0x564583f7ef48 0x564583f499c7 0x7f0a4f9d5b32 0x7f0a4fbf0973 0x7f0a4fbf50bd 0x7f0a4fbf88ee 0x7f0a4fc005c0 0x7f0a4fbf25f5 0x564583f4d050 0x56458403e99d 0x564583fc0fe9 0x564583fbbb0e 0x564583f4e77a 0x564583fbcc9e 0x564583fbbb0e 0x564583f4e77a 0x564583fbcc9e 0x564583fbbb0e 0x564583f4e77a 0x564583fc0e50 0x564583f4e69a 0x564583fc0e50 0x564583fbbb0e 0x564583f4e77a 0x564583fbca45 0x564583fbbb0e 0x564583fbb813 0x564584085592 0x56458408590d 0x5645840857b6
tcmalloc: large alloc 1843200000 bytes == 0x5645f73d0000 @  0x7f0b122cf1e7 0x7f0b0fbb3631 0x7f0b0fc1a680 0x7f0b0fca81a7 0x564583f4d050 0x56458403e99d 0x564583fc0fe9 0x564583f4e69a 0x564583fc0e50 0x564583fbbb0e 0x564583f4e77a 0x564583fbca45 0x564583fbbb0e 0x564583fbb813 0x564584085592 0x56458408590d 0x5645840857b6 0x56458405d103 0x56458405cdac 0x7f0b110b9bf7 0x56458405cc8a
2021-04-01 06:44:44,308 - INFO - util -   num is 600000
tcmalloc: large alloc 1843200000 bytes == 0x